# ARIMA Modeling

> Authors: Matt Brems, Joseph Nelson, Justin Pounders

In [ ]:
# !pip3 install scipy statsmodels --upgrade

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split

import statsmodels.api as sm

# We are required to do this in order to avoid "FutureWarning" issues.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
df = pd.read_csv('./data/global_land_temps.csv')

### Global Temperature Data
> We have [average temperature data](https://data.world/data-society/global-climate-change-data) for the globe from 1750 to 2015.
- `date`: the month the data was measured.
- `avg_temp`: the average global temperature (in degreed Celsius).

In [ ]:
df.head()

In [ ]:
# Set index to be the date column as a DatetimeIndex.
df.set_index()

# Drop the date column from our data.
df.drop()

# Confirm.
df.head()

Let's plot this raw data.

In [ ]:
# Code modified from code written by Matthew Garton.

def plot_series(df, cols=None, title='Title', xlab=None, ylab=None, steps=1):
    
    # Set figure size to be (18, 9).
    plt.figure(figsize=(18,9))
    
    # Iterate through each column name.
    for col in cols:
            
        # Generate a line plot of the column name.
        # You only have to specify Y, since our
        # index will be a datetime index.
        plt.plot(df[col])
        
    # Generate title and labels.
    plt.title(title, fontsize=26)
    plt.xlabel(xlab, fontsize=20)
    plt.ylabel(ylab, fontsize=20)
    
    # Enlarge tick marks.
    plt.yticks(fontsize=18)
    plt.xticks(df.index[0::steps], fontsize=18);

In [ ]:
# Generate a time plot of our data.
plot_series()

<details><summary>What Pandas function may be helpful here?</summary>

- We'll use `.resample()`.
</details>

In [ ]:
# Generate a time plot of our data.
plot_series()

In [ ]:
# Overwrite df.
df =

<details><summary>Check: How would you describe this data?</summary>
    
- **Trend**: The mean is increasing over time and it looks like it speeds up toward the end of the 1960s.
- **Seasonality**: There are fluctuations, but it's tough to see if there are fluctuations on a fixed and known frequency.
- **Autocorrelation**: The data are correlated with one another.
</details>

## ARIMA Model

ARIMA stands for AutoRegressive Integrated Moving Average, and as the name suggests it's composed of three parts (and thus has three parameters): Autoregression, Moving Average, and the integration.

An **autoregressive model** (AR) take a parameter $p$, which is the number of prior timesteps to include. It's effectively the number of lags that we see in our ACF.

**Moving average** (MA) models take previous _error terms_ as inputs. They predict the next value based on deviations from previous predictions. This can be useful for modeling a sudden occurrence — e.g., something going out of stock affecting sales or a sudden rise in popularity.

As with autoregressive models, moving average models have an order term, $q$, and we refer to our model as MA(q). This moving average model is dependent on the last q errors. If we have a time series of sales per week, $y_i$, we can regress each $y_i$ on the last q error terms.

Combining AR and MA gives us an ARMA model, with parameters $p$ and $q$. This model predicts the value of a series. We won't discuss ARMA models, as they tend to be counter productive in time series analysis, and instead we'll move straight on to ARIMA, which integrates both AR and MA using differencing.

In order to fit an ARIMA model, we need to specify all three parameters: $p$, $d$, and $q$.
- $p$ in our autoregressive models (AR) is the number of timesteps to correlate with
- $q$ in our moving average (MA) models is the number of prior error terms to regress on
- $d$ is how much to difference our model

## How to Choose the Right `p` and `q` Parameters
---

In general, it's never a bad idea to choose your parameters based on hold-out testing; that is to say, checking the performance of your model on future time points based on different choices of `p` and `q` for an ARIMA model.

However, you can get a sense for which parameters will work best based on autocorrelation and partial autocorrelation charts.

[This site](https://people.duke.edu/~rnau/411arim3.htm) offers a detailed overview of how to use ACF and PACF to determine your parameters.

Below are some basic general guidelines. Remember that these rules apply to the ACF and PACF plots of differenced time series rather than the original time series (the exception being if your time series is stationary and does not require differencing):

1. If the PACF has a sharp cut-off and the lag-1 ACF value is positive, choose an `AR(x)` term where `x` is the lag in the PACF after the cut-off.
2. If the ACF has a sharp cut-off and the lag-1 ACF value is negative, choose an `MA(x)` term where `x` is the lag in the ACF after the cut-off.
3. If both the ACF and PACF show a gradual decay, an ARMA model is likely appropriate as opposed to AR or MA alone.

Context 1 above corresponds to time series that are "underdifferenced," as indicated by a positive autocorrelation at lat 1. Likewise, Context 2 is "overdifferenced," as indicated by a negative autocorrelation.

In general, you should try to choose an AR or MA model as opposed to an ARMA model. The AR and MA terms can work against each other in the model and create an overly complex representation.

### Parameter $d$: How much do we difference?

$d$ is known as the **differencing parameter**. This controls how much we "difference" our time series by. _(We difference to meet an assumption - we'll talk about this in a minute!)_

Differencing our time series means that instead of fitting a model that predicts our time series $Y_t$ directly, we'll fit a model to $Y'_t$ or $Y''_t$:

$$
\begin{eqnarray*}
Y'_t &=& Y_t - Y_{t-1} \\
Y''_t &=& Y'_t - Y'_{t-1} = Y_t - 2Y_{t-1} + Y_{t-2}
\end{eqnarray*}
$$

So if I wanted to fit a model, my "new" output variable I'm trying to predict would be $Y'_t$ or $Y''_t$.

In [ ]:
# Look at the first 5 values of our series.


In [ ]:
# Look at the first 5 values of our series, differenced once.


In [ ]:
# Look at the first 5 values of our series, differenced twice.


In [ ]:
# Create first_diff_temp and second_diff_temp
# columns in df.
df['first_diff_temp'] =
df['second_diff_temp'] =

[Why would we difference?](https://otexts.com/fpp2/stationarity.html) Well, there is one assumption that is **required** for nearly every time series model: **stationarity**.
- If our time series is stationary, then we do not need to difference and let $d=0$.
- If our time series is not stationary, then we difference either once ($d=1$) or twice ($d=2$). Differenced data often is stationary, so we difference our data, then model that!

#### What is stationarity?

In [ ]:
# Set seed.
np.random.seed(42)

# Generate 200 steps in time from 0 to 15.
t = np.linspace(0, 15, 200)

# Generate a white noise process with Normally distributed noise.
y = 5 * np.random.normal(0, 2, 200)

# Generate plot.
plt.title('A stationary time series!', fontsize=15)
plt.plot(t, y);

Informally, stationarity means that there **aren't systematic changes in our time series over time**.
- Our mean stays the same. (There is no trend.)
- The autocorrelation between $Y_t$ and $Y_{t-k}$ depends only on the size of our lag $k$. (There is no seasonality.)
- A [white noise process](https://stats.stackexchange.com/questions/7070/what-is-a-white-noise-process) is a common example of a stationary time series.

<details><summary>Is our data stationary?</summary>
    
- No! The mean is not constant over time. It looks like the mean increases over time.
- It looks like the fluctuations are more spread out in early years and get closer moving into later years.
</details>

In [ ]:
# Temperature data.
plot_series(df,
            ['avg_temp'],
            title = "Average Temperature (in Celsius)",
            steps = 50)

Again, why do we difference? 
- Differencing allows us to get a stationary time series out of a non-stationary time series.
- This means that we'll be able to fit an ARIMA model to the **differenced** data!

We'll start with a difference of $d = 1$ and iterate upward until we find a value of $d$ that makes our time series stationary.

In [ ]:
# Examine temperature, differenced once.



In [ ]:
# Examine temperature, differenced twice.



How do we know if we let $d=1$ or $d=2$?

#### Checking for Stationarity: the Augmented Dickey-Fuller Test

The [augmented Dickey-Fuller test](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test) is a hypothesis test that tests for stationarity. We assume that our data are not stationary. With enough evidence, we may accept that our data are stationary.

Specifically, the test is as follows:

$$
\begin{eqnarray}
&H_0:& \text{not stationary} \\
&H_A:& \text{stationary}
\end{eqnarray}
$$

<details><summary>Would a small $p$-value or a large $p$-value give us evidence that our time series is stationary?</summary>

- A small $p$-value would give us evidence to reject the null hypothesis, meaning we accept that our time series is stationary.
</details>

In [ ]:
# Import Augmented Dickey-Fuller test.
from statsmodels.tsa.stattools import adfuller

# Run ADF test on original (non-differenced!) data.



In [ ]:
# Code written by Joseph Nelson.

def interpret_dftest(dftest):
    dfoutput = pd.Series(dftest[0:2], index=['Test Statistic','p-value'])
    return dfoutput

In [ ]:
# Run ADF test on original (non-differenced!) data.



<details><summary>At alpha of 0.01, how would we interpret this $p$-value? </summary>
    
- Remember that we compare our $p$-value to $\alpha$, which is usually set at 0.10, 0.05, or 0.01. We'll use $\alpha = 0.01$. Because $p = 0.0498$, it is larger than $\alpha$. Thus, we cannot accept that our series `avg_temp` is stationary.
</details>

##### In order to achieve stationarity, we need to difference!

We'll start with a difference of $d = 1$ to see if the time series `first_diff_temp` is stationary.

In [ ]:
# Run the ADF test on our once-differenced data.



<details><summary>Are our data, after a first-order difference, stationary?</summary>

- Yes! This is stationary.
- With a $p$-value of well below 0.05, we would reject $H_0$ and thus **accept that our first-order differenced data are stationary**.
</details>

- Rather than modeling $Y_t$ directly, we'll model $Y'_t = Y_t - Y_{t-1}$.
- If we generate plots moving forward, then we should generate plots of the **differenced** time series, not the original time series!

### Fit an ARIMA model.

There are three hyperparameters we need to get values for when fitting an ARIMA model.
1. Determine a value of $d$ using the Augmented Dickey-Fuller test.
2. Then, determine values of $p$ and $q$ through GridSearching.

An $ARIMA(p, d, q)$ model is specified by:
- how many differences $d$ we need to calculate in order to achieve stationarity.
- how many lags $p$ we regress $Y_t^{(d)}$ on.
- how many errors $q$ we regress $Y_t^{(d)}$ on.

<details><summary>Check: Recall that $AR$ incorporates information from long-term trends and $MA$ incorporates information from sudden shocks. What are some examples where we might see both long-term trends and sudden fluctuations in our time series data? </summary>

- Stock price data. Stocks increase and decrease over time, but news or other stock changes may have sudden effects on prices. (Similar logic applies to gas or oil prices.)
- Public transportation ridership. While public transportation may see slow changes over time, marketing campaigns, changes in price, or accidents may have a sudden, unforeseen shock on ridership.
- Temperature data over time!
</details>

Depending on our values of $p$, $d$, and $q$, we might refer to these models by slightly different names.

- If $d=0$ and $q=0$, an AR(p) model is specified by how many lags $p$ we regress $Y_t$ on.
- If $d=0$ and $p=0$, an MA(q) model is specified by how many errors $q$ we regress $Y_t$ on.
- If $d=0$, an ARMA(p, q) model is specified by how many lags $p$ and how many errors $q$ we regress $Y_t$ on.

| p | d | q |          Model         |
|:-:|:-:|:-:|:----------------------:|
| 1 | 0 | 0 |  ARIMA(1,0,0) = AR(1)  |
| 0 | 0 | 1 |  ARIMA(0,0,1) = MA(1)  |
| 1 | 0 | 1 | ARIMA(1,0,1) = ARMA(1,1) |
| 1 | 1 | 1 |      ARIMA(1,1,1)      |
| 1 | 1 | 0 |      ARIMA(1,1,0)      |
| 0 | 1 | 1 |      ARIMA(0,1,1)      |


In [ ]:
# Create train-test split.
y_train, y_test = train_test_split()

In [ ]:
# Import ARIMA model.
from statsmodels.tsa.arima_model import ARIMA

## Manual GridSearch

Our strategy is this:
- Let's start AIC at some really, really big number and call it `best_aic`.
- Create variables `best_p` and `best_q` to store our best values of $p$ and $q$.
- Create a nested `for` loop to iterate over our possible values of $p$ and $q$.
    - At every combination of $p$ and $q$, fit an ARIMA model.
    - If the ARIMA model has a better (lower) AIC than `best_aic`, then let's overwrite `best_aic`, `best_p`, and `best_q` with the values of AIC, $p$, and $q$ that our better model is using!
    - If the ARIMA model has a worse (higher) AIC than `best_aic`, do nothing.
- At the end of the `for` loop, tell us which values of $p$ and $q$ give us the best model (lowest AIC).

<details><summary>What should our main concerns be about a manual GridSearch process?</summary>

- The amount of time it takes to fit our models!
- Cross-validation is also more complicated with time series data; you can check out more [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) and [here](https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/).
</details>

<details><summary>What is the purpose of try and except statements?</summary>

- If our code encounters an error, then it will automatically stop. Sometimes this is good (so we can debug) but sometimes this isn't desirable!
- `try` and `except` statements allow us to "try" to do something and, if there's an error, we can just `pass` it so that our code doesn't stop running.
- This isn't always a good thing... errors are usually telling us that something is wrong. But we're going to hack our answer here so that we can check all values of $p$ and $q$ and just not run the model if some values of $p$ and $q$ are invalid.
</details>

In [ ]:
# Starting AIC, p, and q.
best_aic =
best_p =
best_q =

# Use nested for loop to iterate over values of p and q.


        
        # Insert try and except statements.
        try:
            
            # Fitting an ARIMA(p, 1, q) model.
            print(f'')
            
            # Instantiate ARIMA model.
            arima = 
            
            
            # Fit ARIMA model.
            model =

            # Print out AIC for ARIMA(p, 1, q) model.
            print(f'The AIC for ARIMA({p},1,{q}) is: {}')

            # Is my current model's AIC better than our best_aic?
            if
                
                # If so, let's overwrite best_aic, best_p, and best_q.
                best_aic =
                best_p =
                best_q =

        except:
            pass
print()
print()
print('MODEL FINISHED!')
print(f'Our model that minimizes AIC on the training data is the ARIMA({best_p},1,{best_q}).')
print(f'This model has an AIC of {best_aic}.')

> Finding coefficients for an MA model is difficult, because the model requires us to regress on errors, but we don't actually observe these errors. The algorithm for fitting this is complicated and is well beyond the scope of this lesson. [Check more out here](https://www.it.uu.se/research/publications/reports/2006-022/2006-022-nc.pdf). We're using `try` and `except` statements so Python errors don't cause us to stop our model-fitting process!

The model we want to fit is an $ARIMA(1,1,3)$ model. That would look like this:
$$
\begin{eqnarray*}
Y^{(1)}_t &=& AR(1) + MA(3) \\
(Y_t - Y_{t-1}) &=& \beta_0 + \beta_1(Y_{t-1} - Y_{t-2}) + w_1\varepsilon_{t-1} + w_2\varepsilon_{t-2} + w_3\varepsilon_{t-3} + \varepsilon_t \\
\end{eqnarray*}
$$

In [ ]:
# Instantiate best model.
model =


# Fit ARIMA model.
arima =

# Generate predictions based on test set.
preds =

# Plot data.
plt.figure(figsize=(10,6))

# Plot training data.
plt.plot(y_train.index, pd.DataFrame(y_train).diff(), color = 'blue')

# Plot testing data.
plt.plot(y_test.index, pd.DataFrame(y_test).diff(), color = 'orange')

# Plot predicted test values.
plt.plot(y_test.index, preds, color = 'green')

plt.title(label = 'Once-Differenced Global Mean Temperature with ARIMA(1, 1, 3) Predictions', fontsize=16)
plt.show();

<details><summary>What do these predicted values represent?</summary>

- These predicted values represent $Y'_t$, which is the once-differenced mean temperature data.
- The first observation represents the "change in average temperature" in 1752.
- The second observation represents the "change in average temperature" in 1753.
- And so on.
- If you want to "undo" the predicted values to forecast forward, you can take the original $Y$ value at time $t-1$ and add that to $Y'_t$.

$$Y'_t = Y_t - Y_{t-1} \Rightarrow Y_t = Y'_t + Y_{t-1}$$
</details>

## A Quick Note about SARIMA or SARIMAX
---
With ARIMA we've captured trend and difference, but what about seasonality? None of the methods we've talked about yet include it. 

Enter seasonal ARIMA (or SARIMA). (The 'X' at the end is a python thing that indicates the model also supports exogenous variables.) It takes all the parameters of the base ARIMA model, with additional parameters (denoted $P, D, Q$ and $m$) to account for lags in season.

So we denote the whole thing as: $SARIMA(p,d,q)(P,D,Q)m$ where $m$ indicates the number of timesteps in a season.

A P=1 would make use of the first seasonally offset observation in the model, e.g. $t-(m*1)$ or $t-12$. A P=2, would use the last two seasonally offset observations $t-(m * 1)$, $t-(m * 2)$.

Similarly, a D of 1 would calculate a first order seasonal difference and a Q=1 would use a first order errors in the model (e.g. moving average).

More information about SARIMA can, of course, be found [in the documentation](https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html).

And [this tutorial](https://machinelearningmastery.com/sarima-for-time-series-forecasting-in-python/) from Dr. Jason Brownlee has a good overview of how to implement SARIMA in python.

## Takeaways
- ARIMA models are among the most common approaches to time series modeling.
- Highly flexible; it can model time series with varying characteristics.
    - It takes information from both long-term trends and sudden shocks!
- Can easily be extended into more advanced models.
    - Seasonal ARIMA.
    - ARIMA with eXogenous Predictors. (Independent X variables.)
    - Vector ARIMA models. (Multiple Y variables simultaneously.)
- Tends to perform well with moderate amounts of data, but may be outperformed by deep learning methods (RNN) with lots of data. (Side note: It can be hard to get lots of time series data!)
- ARIMA models are best suited for short-term forecasts, but very quickly will start predicting the mean. Some of the extensions to ARIMA models can handle this better.

**Forecasting time series is exceptionally difficult to do well!** We're using the past to predict the future, and there aren't guarantees that the future is necessarily going to be reflective of the past. There's more uncertainty in time series than we'd see in "traditional" model-building.

### Interview Questions

<details><summary>What are examples of time series data?</summary>

_(Answers will vary.)_
- Any data that is gathered over time is considered time series data.
- Examples include an individual's maximum heart rate per minute, average temperature per hour, number of units sold per day, amount of rainfall per month, and GDP per quarter.
</details>

<details><summary>What are some disadvantages to time series analysis methods?</summary>

_(Answers will vary.)_
- Forecasting into the future is difficult! There are more sources of uncertainty than in traditional, non-temporal machine learning.
- The amount of data gathered over time is often smaller than the amount of data in non-temporal machine learning.
- Forecasts may only be meaningful/informative for a few time periods.
- [Significant changes](gov/pmc/articles/PMC5464762/) may cause certain periods of data to be irrelevant/less relevant to solving problem. (e.g. data prior to Great Recession may not be impactful when forecasting economic data after 2009, data prior to COVID may not be meaningful when predicting consumer behavior after May 2020.)
</details>